In [1]:
# Minkowski bound not always tight
from math import sqrt

# liste croissante de k
Ks = [1,2,5,10,50,100,500,1000]

for k in Ks:
    # définir les vecteurs b1,b2
    b1 = vector(QQ, [k, 0])
    b2 = vector(QQ, [k, QQ(1)/k])
    
    # construire la matrice B
    B = matrix(QQ, [[b1[0], b2[0]], [b1[1], b2[1]]])
    assert B.determinant() == 1 # par construction
    
    # vecteur v_k = b2 - b1
    vk = b2 - b1
    assert vk.norm() == QQ(1)/k # par construction
    
    # afficher résultats pour ce k
    print(f"k = {k}, vk = {vk}, ||vk|| = {float(vk.norm())}")
    
    
# comparaison avec la borne de Minkowski en dimension 2 (det=1)
minkowski_bound = sqrt(2) * (1**(1/2))  # = sqrt(2) 
print("\nMinkowski bound for n=2, det=1 : ", minkowski_bound)

k = 1, vk = (0, 1), ||vk|| = 1.0
k = 2, vk = (0, 1/2), ||vk|| = 0.5
k = 5, vk = (0, 1/5), ||vk|| = 0.2
k = 10, vk = (0, 1/10), ||vk|| = 0.1
k = 50, vk = (0, 1/50), ||vk|| = 0.02
k = 100, vk = (0, 1/100), ||vk|| = 0.01
k = 500, vk = (0, 1/500), ||vk|| = 0.002
k = 1000, vk = (0, 1/1000), ||vk|| = 0.001

Minkowski bound for n=2, det=1 :  1.4142135623730951


In [2]:
# Testing inclusion of lattices
def is_sublattice_simple(B1, B2):
    """
    B1, B2: matrices n x n sur QQ.
    Retourne (True, X) si L1 = <B1>_Z est inclus dans L2 = <B2>_Z et X = B2^{-1}*B1 est entier.
    Sinon (False, None).
    """
    B1q = matrix(QQ, B1)
    B2q = matrix(QQ, B2)
    if B2q.det() == 0:
        raise ValueError("B2 singulier")

    X = B2q.inverse() * B1q
    
    # vérifier intégralité : chaque entrée doit avoir denominator == 1
    if not all(X[i,j].denominator() == 1 for i in range(X.nrows()) for j in range(X.ncols())):
        return (False, None)
            
    # renvoyer la matrice entière
    X_int = matrix(ZZ, X)
    return (True, X_int)


print("=== Test 1: Réseau 2Z × 2Z inclus dans Z^2 ===")
B1 = matrix(QQ, [[2,0],[0,2]])   # B1 génère 2Z x 2Z
B2 = matrix(QQ, [[1,0],[0,1]])   # B2 = I2

result, X = is_sublattice_simple(B1,B2)
print(f"{result}, la matrice X = B2^(-1) * B1 =")
print(X); print()

print("=== Test 2: B2 = B1 * U avec U unimodulaire ===")
B1 = matrix(QQ, [[1.5, 0.2, 3.1], [0.0, -2.3, 4.4], [2.2, 1.1, 0.5]])
U = matrix(QQ, [[1, 2, 0], [0, 1, 1], [0, 0, 1]]) # unimodular matrix
B2 = B1 * U

result, X = is_sublattice_simple(B1,B2)
assert X == U.inverse(), "Erreur: X != U^(-1)"
print(f"{result}, la matrice X = B2^(-1) * B1 =")
print(X); print()

print("=== Test 3: B1 avec déterminant != 1 ===")
B1 = matrix(QQ, [[2, 0], [0, 1]]) # (det != 1)
B2 = matrix(QQ, [[1, 0], [0, 1]]) # Z^2
result, X = is_sublattice_simple(B1,B2)

print(f"{result}, la matrice X = B2^(-1) * B1 =")
print(X); print()

=== Test 1: Réseau 2Z × 2Z inclus dans Z^2 ===
True, la matrice X = B2^(-1) * B1 =
[2 0]
[0 2]

=== Test 2: B2 = B1 * U avec U unimodulaire ===
True, la matrice X = B2^(-1) * B1 =
[ 1 -2  2]
[ 0  1 -1]
[ 0  0  1]

=== Test 3: B1 avec déterminant != 1 ===
True, la matrice X = B2^(-1) * B1 =
[2 0]
[0 1]



In [3]:
# Finding a short vector in a lattice of dimension 2
B = Matrix(ZZ, 2, [1, 0, 402, 1009])
assert B.nrows() == 2 and B.ncols() == 2
assert B.det() != 0

print("B =")
print(B)
print(f"Déterminant: det(B) = {B.det()}")
print(f"Normes: ||b1|| = {vector(B.column(0)).norm().n():.3f}, ||b2|| = {vector(B.column(1)).norm().n():.3f}")


BLLL = B.LLL()
print("\nB.LLL() =")
print(BLLL)

print("\nB.LLL().transpose() =")
print(BLLL.transpose())

Bt_LLLt = B.transpose().LLL().transpose()
print("\nB.transpose().LLL().transpose() =")
print(Bt_LLLt)

# Lagrange-Gauss, on suppose B 2x2 sur QQ
def lagrange_gauss_from_matrix(B, max_iter):
    """
    Algorithme de Lagrange-Gauss pour réduire une base 2x2
    
    Args:
        B: matrice 2x2 sur ZZ
        max_iter: nombre maximum d'itérations
    
    Returns:
        Matrice 2x2 avec base réduite en colonnes
    """
    # Extraire les vecteurs colonnes
    b1 = vector(QQ, [B[0,0], B[1,0]])
    b2 = vector(QQ, [B[0,1], B[1,1]])
    
    iteration = 0
    while True:
        if iteration > max_iter:
            raise RuntimeError("Trop d'itérations dans Lagrange-Gauss")
        
        # s'assurer b1 est le plus court
        if b1.norm() > b2.norm():
            b1, b2 = b2, b1
        
        # Calculer le coefficient de Gram-Schmidt et l'arrondir
        mu = b1.dot_product(b2) / b1.dot_product(b1)
        m = Integer(round(mu))
        
        # Réduction
        b2 = b2 - m*b1
        if b2.norm() >= b1.norm():
            break
        iteration += 1
     
     # renvoie matrice dont colonnes sont la base réduite
    return matrix(ZZ, [list(b1), list(b2)]).transpose()

B_lg = lagrange_gauss_from_matrix(B, 100)
assert B.det() == B_lg.det(), "Erreur: déterminant non conservé"

print("\nBase obtenue par Lagrange-Gauss: ")
print(B_lg)
print(f"Déterminant: det(B_lg) = {B_lg.det()}")
print(f"Normes: ||b1|| = {vector(B_lg.column(0)).norm().n():.3f}, ||b2|| = {vector(B_lg.column(1)).norm().n():.3f}")

B =
[   1    0]
[ 402 1009]
Déterminant: det(B) = 1009
Normes: ||b1|| = 402.001, ||b2|| = 1009.000

B.LLL() =
[   1    0]
[   0 1009]

B.LLL().transpose() =
[   1    0]
[   0 1009]

B.transpose().LLL().transpose() =
[ -5 -93]
[  8 -53]

Base obtenue par Lagrange-Gauss: 
[ -5 -93]
[  8 -53]
Déterminant: det(B_lg) = 1009
Normes: ||b1|| = 9.434, ||b2|| = 107.042


In [4]:
# Lattice minima
B = matrix(ZZ, [[6, 12, -5], [0, 2, 7], [0, 0, 3]])
print("B ="); print(B)

# Vecteurs de la base
c1 = B.column(0); c2 = B.column(1); c3 = B.column(2)
print(f"c1 = {c1}, ||c1|| = {float(c1.norm())}")
print(f"c2 = {c2}, ||c2|| = {float(c2.norm())}")
print(f"c3 = {c3}, ||c3|| = {float(c3.norm())}\n")

# 4.1
print("Determinant et borne de Minkowski:")
detB = B.determinant()
print("det(B) =", detB)

minkowski_bound = sqrt(3) * (detB ** (1/3))
print(f"Minkowski bound (sqrt(3)*det^(1/3)) ≈ {float(minkowski_bound)}\n")
assert minkowski_bound < 6, "La borne de Minkowski n'est pas vérifié"

# 4.3
print("Montrer lambda1 = 2 en exhibant un vecteur de norme 2:")
v = -2*c1 + c2
print(f"v = -2*c1 + c2 = {v}, ||v|| = {float(v.norm())}\n")
assert v.norm() == 2

# 4.4
print("Montrer lambda3 <= 6 en exhibant 3 vecteurs lin. indép. de norme <= 6:")
v2 = c1
v3 = 3*c1 - c2 + c3
print(f"v1 = {v}, ||v1|| = {float(v.norm())}") # ||v2|| = 2

print(f"v2 = {v2}, ||v2|| = {float(v2.norm())}") # ||v1|| = 6
assert v2.norm() <= 6

print(f"v3 = {v3}, ||v3|| = {float(v3.norm())}\n") # ||v3|| < 6
assert v3.norm() <= 6

M = matrix(ZZ, [list(v), list(v2), list(v3)]).transpose()   # colonnes = v1,v2,v3
print("rang([v1,v2,v3]) =", M.rank())
assert M.rank() == 3

B =
[ 6 12 -5]
[ 0  2  7]
[ 0  0  3]
c1 = (6, 0, 0), ||c1|| = 6.0
c2 = (12, 2, 0), ||c2|| = 12.165525060596439
c3 = (-5, 7, 3), ||c3|| = 9.1104335791443

Determinant et borne de Minkowski:
det(B) = 36
Minkowski bound (sqrt(3)*det^(1/3)) ≈ 5.719105757981619

Montrer lambda1 = 2 en exhibant un vecteur de norme 2:
v = -2*c1 + c2 = (0, 2, 0), ||v|| = 2.0

Montrer lambda3 <= 6 en exhibant 3 vecteurs lin. indép. de norme <= 6:
v1 = (0, 2, 0), ||v1|| = 2.0
v2 = (6, 0, 0), ||v2|| = 6.0
v3 = (1, 5, 3), ||v3|| = 5.916079783099616

rang([v1,v2,v3]) = 3


In [5]:
# Babai round-off algorithm
def babai_round(B, t):
    """
    Babai round-off en flottant :
    - B : matrice n x n (colonnes = b_i)
    - t : vecteur cible dans R^n
    Retourne (s, residual, x, xr) où:
    - x: coefficients réels (B^{-1} * t)
    - xr: coefficients entiers arrondis
    - s: point du réseau B * xr
    - residual: vecteur résiduel t - s
    """
    # conversion dans RR
    Br = matrix(RR, B)
    tr = vector(RR, t)
    
    Xin = Br.inverse() * tr
    
    # arrondi des coefficients
    xr = vector(ZZ, [Integer(round(Xin[i])) for i in range(len(Xin))])
    
    # reconstruction du point s
    s_real = Br * xr            
    
    # tenter de convertir s en vecteur entier si les composantes sont (pratiquement) entières
    s_components = []
    is_integral = True
    
    for component in s_real:
        rounded_comp = round(float(component))
        if abs(float(component) - rounded_comp) > 1e-8:
            is_integral = False
            break
        s_components.append(Integer(rounded_comp))
    
    s = vector(ZZ, s_components) if is_integral else s_real
    
    # Calcul du résiduel
    residual = tr - vector(RR, s)
    return (s, residual, x, xr)

In [6]:
B = matrix(ZZ, [[6, 12, -5], [0, 2, 7], [0, 0, 3]])
print("B ="); print(B); print()

# borne: ||t-s|| <= 1/2 * n * max_i ||b_i||
print("Borne: ||t - s|| ≤ (1/2) * n * max_i ||b_i||")
n = B.ncols()
max_norm = max([float(B.column(i).norm()) for i in range(n)])
bound = 0.5 * n * max_norm

print(f"n = {n}")
print(f"max_i ||b_i|| = {max_norm}")
print(f"Borne = (1/2) × {n} × {max_norm} = {bound}\n")

# duale
Bhat = B.inverse().transpose()
print("Base duale B^⊥ (colonnes = b_i^⊥):")
print(Bhat)

test_vectors = [
    vector(RR, [0.1, 0.2, 0.3]),
    vector(RR, [5.7, -1.3, 2.2]),
    vector(RR, [12.34, 7.89, -3.21])
]
for i, t in enumerate(test_vectors, 1):
    print(f"\n--- Test {i} ---")
    print(f"t = {t}")
    
    s, resid, x, xr = babai_round(B, t)
    print("x (coeffs)    =", x)
    print("xr (rounded)  =", xr)
    print("s = B*xr      =", s)
    print("||t - s||     =", float(resid.norm()), " <= bound? ", (resid.norm() <= bound))

B =
[ 6 12 -5]
[ 0  2  7]
[ 0  0  3]

Borne: ||t - s|| ≤ (1/2) * n * max_i ||b_i||
n = 3
max_i ||b_i|| = 12.165525060596439
Borne = (1/2) × 3 × 12.165525060596439 = 18.2482875908947

Base duale B^⊥ (colonnes = b_i^⊥):
[  1/6     0     0]
[   -1   1/2     0]
[47/18  -7/6   1/3]

--- Test 1 ---
t = (0.100000000000000, 0.200000000000000, 0.300000000000000)
x (coeffs)    = x
xr (rounded)  = (1, 0, 0)
s = B*xr      = (6, 0, 0)
||t - s||     = 5.91100668245266  <= bound?  True

--- Test 2 ---
t = (5.70000000000000, -1.30000000000000, 2.20000000000000)
x (coeffs)    = x
xr (rounded)  = (8, -3, 1)
s = B*xr      = (7, 1, 3)
||t - s||     = 2.7604347483684517  <= bound?  True

--- Test 3 ---
t = (12.3400000000000, 7.89000000000000, -3.21000000000000)
x (coeffs)    = x
xr (rounded)  = (-14, 8, -1)
s = B*xr      = (17, 9, -3)
||t - s||     = 4.7949765380030795  <= bound?  True


In [7]:
# calcul approché de lambda1 sur une petite fenêtre
def compute_lambda1_enum(B, R=4):
    """
    Calcul approché de λ1 par énumération sur une fenêtre [-R, R]
    Args:
        B: matrice de base du réseau
        R: rayon de la fenêtre de recherche    
    Returns:
        (lambda1, coeffs) où lambda1 est la norme minimale et coeffs les coefficients
    """
    n = B.ncols()
    min_norm = +Infinity
    best_coeffs  = None
    
    # Énumération sur tous les coefficients dans [-R, R]^n
    for coeffs in cartesian_product_iterator([range(-R, R+1)]*n):
        # Ignorer le vecteur nul
        if all(c == 0 for c in coeffs):
            continue
        
        # Construire le vecteur v = somme(coeffs[i] * colonne_i)
        v = sum([coeffs[i]*B.column(i) for i in range(n)])
        norm_v = v.norm()
        
        if norm_v < min_norm:
            min_norm = norm_v; best_coeffs = tuple(coeffs)
    
    return float(min_norm), best_coeffs 

enum = 3
lambda1, min_coeffs = compute_lambda1_enum(B, R=enum)
print(f"Énumération sur la fenêtre [-{enum}, {enum}]^3")
print(f"λ1 (approximation) = {lambda1}")
print(f"Coefficients du vecteur minimal: {min_coeffs}")

# pour B on sait lambda1 = 2
assert lambda1 == 2

Énumération sur la fenêtre [-3, 3]^3
λ1 (approximation) = 2.0
Coefficients du vecteur minimal: (-2, 1, 0)


In [8]:
# Construire v = -2*c1 + c2 = (0,2,0) dans L:
v = sum([min_coeffs[i]*B.column(i) for i in range(3)])
print(f"v = {v}, ||v|| = {float(v.norm())}\n")

# Paramètres pour la perturbation
gamma = 4
e_norm_bound = float(lambda1 / gamma)
print(f"Paramètre γ = {gamma}")
print(f"Borne sur ||e||: λ1/γ = {lambda1}/{gamma} = {e_norm_bound}\n")

# Construction du vecteur avec perturbation e
e = vector(RR, [0.9*e_norm_bound, 0.0, 0.0])
print(f"e = {e}")
print(f"||e|| = {float(e.norm())} < {e_norm_bound}\n")

# Vecteur cible t = v + e
t = vector(RR, v) + e
print(f"Vecteur cible: t = v + e = {t}")

v = (0, 2, 0), ||v|| = 2.0

Paramètre γ = 4
Borne sur ||e||: λ1/γ = 2.0/4 = 0.5

e = (0.450000000000000, 0.000000000000000, 0.000000000000000)
||e|| = 0.45 < 0.5

Vecteur cible: t = v + e = (0.450000000000000, 2.00000000000000, 0.000000000000000)


In [9]:
# afficher <e, b_i^⊥> pour vérifier condition
print("Conditions: |<e, b_i^⊥>| < 1/2 pour tout i")
for i in range(n):
    bi_dual = Bhat.column(i)
    projection = float(e.dot_product(vector(RR, bi_dual)))
    
    print(f"i = {i+1}: <e, b_{i+1}^⊥> = {projection}")
    print(f"\t|{projection}| < 0.5? {abs(projection) < 0.5}")

Conditions: |<e, b_i^⊥>| < 1/2 pour tout i
i = 1: <e, b_1^⊥> = 0.075
	|0.075| < 0.5? True
i = 2: <e, b_2^⊥> = 0.0
	|0.0| < 0.5? True
i = 3: <e, b_3^⊥> = 0.0
	|0.0| < 0.5? True


In [10]:
# appliquer Babai
s, resid, x, xr = babai_round(B, t)
print("Appliquer Babai sur t = v + e\n")
s, residual, x, x_rounded = babai_round(B, t)

print(f"Vecteur trouvé s = {s}")
print(f"Vecteur original v = {v}")
print(f"s == v? {s == v}")
print(f"||t - s|| = {float(residual.norm())}\n")

if s == v:
    print("Babai a récupéré v.")
else:
    print("Babai n'a pas récupéré v - revoir gamma / condition |<e,b^>| < 1/2.")

Appliquer Babai sur t = v + e

Vecteur trouvé s = (0, 2, 0)
Vecteur original v = (0, 2, 0)
s == v? True
||t - s|| = 0.45

Babai a récupéré v.


In [11]:
# Minkowski’s bound for kernel lattices
import random
from itertools import product

def make_random_A(q, m, r, seed=None):
    """
    Construit A dans (Z/qZ)^{m x r} de rang r. 
    Retourne une matrice ZZ avec représentants dans [0,q-1].
    """
    rng = random.Random(int(seed) if seed is not None else None)
    
    while True:
        # Générer des données aléatoires
        data = [rng.randrange(q) for _ in range(m * r)]
        Aq = Matrix(GF(q), m, r, data)
        
        if Aq.rank() == r:
            # représenter par des entiers 0..q-1 dans ZZ
            return Matrix(ZZ, m, r, [int(a) for a in Aq.list()])

def find_collision_modA(A, q, B):
    """
    Parcours exhaustif de S_B = {x in Z^m : ||x||_inf <= B}.
    Retourne (x1, x2, iterations) à la première 
    collision x1^T A == x2^T A (mod q).
    """
    m = A.nrows(); r = A.ncols()
    images = {} # dict: key -> premier x rencontré
    
    iteration = 0    
    for coeffs in product(range(-B, B+1), repeat=m):
        iteration += 1
        
        # Calculer x^T A mod q
        image = []
        for j in range(r):
            component_sum = sum(coeffs[i] * A[i, j] for i in range(m))
            image.append(component_sum % q)
        
        key = tuple(image)
        
        # Vérifier s'il y a collision
        if key in images:
            return images[key], coeffs, iteration
        images[key] = coeffs
        
    return None, None, iteration

def construct_z_and_check(x1, x2, A, q):
    """Construit z = x1-x2, calcule ||z||_inf, ||z||_2 et vérifie z^T A ≡ 0 (mod q)."""
    m = A.nrows()
    z = tuple(x1[i] - x2[i] for i in range(m))
    
    # Calcul des normes
    norm_inf = max(abs(zi) for zi in z)
    norm2 = math.sqrt(sum(zi*zi for zi in z))
    
    # Vérification que z^T A = 0 (mod q)
    residuals = []
    for j in range(A.ncols()):
        s = sum(z[i] * A[i, j] for i in range(m))
        residuals.append(s % q)
    
    is_zero_mod = all(r == 0 for r in residuals)
    return z, norm_inf, norm2, is_zero_mod, tuple(residuals)

def demo_exhaustive(q, m, r, seed=None):
    q, m, r = int(q), int(m), int(r)
    print(f"Paramètres : q={q}, m={m}, r={r}\n")
    
    # Construction de la matrice A
    A = make_random_A(q, m, r, seed=seed)
    print(f"Matrice A mod {q} :")
    print(A); print()
    
    # Calcul de B et du cardinal de S_B
    B = math.ceil(0.5 * (q ** (r / m)))
    SB_size = (2*B + 1) ** m
    print(f"B = ⌈(1/2) × {q}^({r}/{m})⌉ = {B}  => |S_B| = (2B+1)^m = {SB_size}\n")
    print(f"{q}^{r} = {q**r} => |S_B| > {q}^{r} ? {SB_size > q**r}\n")

    # lancer la recherche exhaustive d'une collision
    print(f"Recherche exhaustive d'une collision modulo {q} dans S_{B}...")
    x1, x2, iters = find_collision_modA(A, q, B)
    
    if x1 is None:
        print("Aucune collision trouvée.")
        return
    
    print(f"Collision trouvée après {iters} itérations.")
    print(f"x1 = {x1}")
    print(f"x2 = {x2}\n")

    # Construire z
    z, norm_inf, norm2, is_zero_mod, residuals = construct_z_and_check(x1, x2, A, q)
    print("z = x1 - x2 =", z)
    print("||z||_inf =", norm_inf)
    print("||z||_2   = ", norm2)
    print(f"z^T A mod q = {residuals} => z^T A = 0 (mod {q})? {is_zero_mod}\n")
    
    # bornes : sup et Minkowski
    minkowski = math.sqrt(float(m)) * (float(q) ** (float(r) / float(m)))
    sup = minkowski + 2.0 * math.sqrt(float(m))
    
    print(f"Borne de Minkowski : sqrt({m})*{q}^({r}/{m}) = {minkowski}")
    print(f"Borne supérieur : sqrt({m})*{q}^({r}/{m}) + 2*sqrt({m}) = {sup}")
    print("Comparaison : ||z||_2 <= sup ? ->", norm2 <= sup)

  
demo_exhaustive(q=101, m=4, r=2, seed=None)

Paramètres : q=101, m=4, r=2

Matrice A mod 101 :
[75 65]
[92 15]
[97 95]
[32 13]

B = ⌈(1/2) × 101^(2/4)⌉ = 6  => |S_B| = (2B+1)^m = 28561

101^2 = 10201 => |S_B| > 101^2 ? True

Recherche exhaustive d'une collision modulo 101 dans S_6...
Collision trouvée après 2098 itérations.
x1 = (-6, -6, -6, -6)
x2 = (-6, 6, -1, -2)

z = x1 - x2 = (0, -12, -5, -4)
||z||_inf = 12
||z||_2   =  13.601470508735444
z^T A mod q = (0, 0) => z^T A = 0 (mod 101)? True

Borne de Minkowski : sqrt(4)*101^(2/4) = 20.09975124224178
Borne supérieur : sqrt(4)*101^(2/4) + 2*sqrt(4) = 24.0997512422418
Comparaison : ||z||_2 <= sup ? -> True
